In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from src.utilsgru import crear_ventanas_temporales, train_model
from models.gru import GRUModel
from models.gru import GRUDataset
from torch.utils.data import DataLoader

In [ ]:
df = pd.read_csv("data/modelado/ds_modelado.csv")

# Posible separacion ene l dataset modelado, no aparece de forma clara las fechaas, quizas interpreta mal
df["estacion_del_anio"] = df["estacion_del_anio"].str.lower()
df["fecha_origen_recorrido"] = pd.to_datetime(df["fecha_origen_recorrido"])
df["anio"] = df["fecha_origen_recorrido"].dt.year

# Filtrar por verano únicamente
df_verano = df[df["estacion_del_anio"] == "verano"].copy()


In [ ]:
#separo en train y val
df_train = df_verano[df_verano["anio"].between(2020, 2023)]
df_test = df_verano[df_verano["anio"] == 2024] #o sea de primero de enero hasta 21 marzo 

In [ ]:
features = [col for col in df.columns if col not in ["N_arribos_intervalo", "anio"]] #todas menos el target y el año (el )
target = "N_arribos_intervalo"

In [ ]:
#paso todo a 
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(df_train[features])
y_train = df_train[target].values

X_test_scaled = scaler.transform(df_test[features])
y_val = df_test[target].values

ventana = 6 #la ventana temporal de tamanio 6. para predecir el valor instante t entonces usas t - 6, t - 5, ..., t - 1
X_train_seq, y_train_seq = crear_ventanas_temporales(X_train_scaled, y_train, ventana)
X_test_seq, y_test_seq = crear_ventanas_temporales(X_test_scaled, y_val, ventana)


In [ ]:
train_dataset = GRUDataset(X_train_seq, y_train_seq)
test_dataset = GRUDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Entrenar el modelo
model = GRUModel(input_dim=X_train_seq.shape[2], hidden_dim=64)
train_model(model, train_loader, test_loader, epochs=10, lr=0.001)